In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
import operator
import pandas as pd
import numpy as np
import os
import pprint
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import pandasql as sqldf

Ziel dieses Notebooks ist es die Hyperparameter Lernrate und Epochenanzahl für das gegebene Neuronale Netz zu optimieren. Hierfür wird eine Klasse verwendet, die in gegebenen Intervallen verschiedene Kombinationen dieser Parameter geordnet ausprobiert (kein random search) und das beste Resultat, basierend auf übergebenen Trainingsdaten zurück gibt. 

In [2]:
def load_data(directory = 'sliced_data'):
    '''
        Funktion, die die aufbereiteten/vorbereiteten Daten aus existierenden CSV Dateien 
        einliest und je nach Vollständigkeit in zwei Dictionaries abspeichert,
        geschlüsselt nach der jeweiligen RaceId
    '''
    #Ids von Rennen, die als Regenrennen identifiziert wurden
    rain_id = [847,861,879,910,914,934,942,953,957,967,950,982]
    if os.path.exists(directory):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/'+directory):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        sliced_races = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv(directory+'/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv(directory+'/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            #print(df.head())
            f = int(file.split('_')[-1].split('.')[0])
            df["rain"] = 0
            #bias wird hinzugefügt
            #df['bias'] = 1
            #setzen der regenkomponente auf 1 für regenrennen
            if list(df["raceId"])[0] in rain_id:
                df["rain"] = 1
            sliced_races[f] = df
        print('Einlesen der sliced Dateien erfolgreich')
    else:
        raise ('sliced Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
        
    if os.path.exists('split_data'):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/split_data'):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        split_by_race = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            f = int(file.split('_')[-1].split('.')[0])
            #hinzufügen eines bias
            #df['bias'] = 1
            split_by_race[f] = df
        print('Einlesen der split Dateien erfolgreich')
    else:
        raise('split Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
        
    return sliced_races, split_by_race


def train_dev_test(data_dict, train_p = 0.7, dev_p = 0.2, test_p = 0.1, nogo_columns = []):
    
    if round(train_p+dev_p+test_p,1) !=1.0:
        raise ValueError ('No valid train/dev/test distribution')
    
    '''
        Daten werde in einem Dictionary übergeben, Dataframes werden in dieser 
        Funktion geshuffled und dann in einen Traindatensatz und in einen 
        Testdatensatz aufgeteilt.
    '''
    #aufteilen in train, dev, test counter
    train_count = round(len(data_dict.keys())*train_p, 0)
    dev_count = train_count+round(len(data_dict.keys())*dev_p,0)
    test_count = len(data_dict.keys())-(train_count+dev_count)
    
    #shufflen der übergebenen Daten
    keys = list(data_dict.keys())
    random.shuffle(keys)
    data_shuffled = {}
    for key in keys:
        data_shuffled[key] = data_dict[key]
        
    #erzeugen separater train,dev,test dictionaries
    train = {}
    dev = {}
    test = {}
    c = 0
    
    #daten sollen nicht in tensoren umgewandelt werden
    for id, df in data_shuffled.items():
        #entfernen nicht gewollter spalten aus dataframe
        cols = [col for col in df.columns if col not in nogo_columns]
        df = df[cols]
        if c < train_count:
            train[id] = df
        elif c >= train_count and c < dev_count:
            dev[id] = df
        else:
            test[id] = df
        c += 1
                
    return train, dev, test

def to_tensor(train_data, dev_data, test_data, nogo_columns = []):
    train = []
    train_ = {}
    dev = []
    dev_ = {}
    test = []
    test_ = {}
    
    for id, race in train_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            stop_sum = np.sum(temp['stop_binary'])#anzahl boxenstops
            temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
            temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            #stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
            temp_x = temp_x.tail(1)
            temp_x['stop_binary'] = stop_sum
            x_tensor = torch.tensor(temp_x[cols].values)#torch.tensor(temp_x.values)
            #temp_x = x_tensor.float()
            train.append((x_tensor, [temp_y[0]]))
        train_[id] = train
        train = []
    for id, race in dev_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            stop_sum = np.sum(temp['stop_binary'])#anzahl boxenstops
            temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
            temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            #stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
            temp_x = temp_x.tail(1)
            temp_x['stop_binary'] = stop_sum
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            #temp_x = x_tensor.float()
            dev.append((x_tensor, [temp_y[0]]))
        dev_[id] = dev
        dev = []
    for id, race in test_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            stop_sum = np.sum(temp['stop_binary'])#anzahl boxenstops
            temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
            temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            #stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
            temp_x = temp_x.tail(1)
            temp_x['stop_binary'] = stop_sum
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            #temp_x = x_tensor.float()
            test.append((x_tensor, [temp_y[0]]))
        test_[id] = test
        test = []
        
    return train_,dev_,test_
        
#torch.tensor(df.values)
def train_test (data_dict, test_num = 5, nogo_columns = []):
    train_data = []
    test_data = []
    test_final = {}
    temp_y_podium = []
    test_races = list(data_dict.keys())
    random.shuffle(test_races)
    test_races = test_races[0:test_num]
    for key, value in data_dict.items():
        helper = key
        for did in value.driverId.unique():
            temp = value.where(value.driverId == did).dropna(how = "all")
            if list(temp["podium_position"])[0] < 0: #Top x finish positions
                pp = 1
            else:
                if key in test_races:
                    temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                    temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                    #temp_y = temp_y[0]
                    cols = [col for col in temp.columns if col not in nogo_columns]
                    temp_x = temp[cols]
                    stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                    temp_x = temp_x.tail(1)
                    temp_x['stop_binary'] = stops
                    x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                    #temp_x = x_tensor.float()
                    test_data.append((x_tensor, [temp_y[0]]))
                else:
                    temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                    temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                    #temp_y = temp_y[0]
                    cols = [col for col in temp.columns if col not in nogo_columns]
                    temp_x = temp[cols]
                    stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                    temp_x = temp_x.tail(1)
                    temp_x['stop_binary'] = stops
                    x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                    #temp_x = x_tensor.float()
                    train_data.append((x_tensor, [temp_y[0]]))
        if key in test_races:
            test_final[key]=test_data
        test_data = []
    random.shuffle(train_data)
    #random.shuffle(test_data)
    #test_data = train_data[len(train_data)-100:]
    train_data = train_data#[0:len(train_data)-100]
            
            #break
            #for i, row in temp.iterrows():
            
    return train_data, test_final

Einlesen der Dateien als Dataframes, welche in zwei Dictionaries gespeichert werden:<br>
- Attribut directory verweißt auf den Ordnernamen der sliced Dateien, Default ist 'sliced_data'
- für die split_by_race Daten sollte kein separates Directory angegeben werden müssen, da es keinen Unterschied zwischen den Rennen gibt! (Nur bei Slicing: Wie viele Prozent des Rennens möchte ich? ergibt ein anderes Directory Sinn)

In [3]:
sliced_races, split_by_race = load_data(directory = 'sliced_data_50_neueForm')
#temp_x = load_data(directory = 'sliced_data_50_neueForm')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


Aufteilen des Datensatzes in einen Trainings-, einen Development- und einen Testteil mit Hilfe der Funktion train_dev_test(). Standardmäßig wird der Datensatz zu 70% in Trainings-, zu 20% in Dev- und zu 10% in Testdaten aufgeteilt. Die Verteilung kann mit den Parametern train_p,dev_p,test_p die der Funktion train_dev_test() übergeben werden können, angepasst werden. Wichtig ist bloß, dass insgesamt train_p,dev_p und test_p 1 ergeben. Die Datenbeispiele aus sliced_races werden vor dem Aufteilen geshuffled. Die Funktion train_dev_test() gibt die Datenbeispiele dann wieder in Dictionary Form zurück, wobei jeder RaceId ein DataFrame mit Renndaten zugeordnet wird. Im nächsten Schritt werden die drei Datensammlungen der Funktion to_tensor() übergeben, die nicht gewünschte Spalten aus den Datensätzen entfernt (nogo_columns) und diese dann in Tensoren abspeichert, die wiederrum  ihrem Targetvalue (podiums_position) zugeordnet werden, um die Überprüfung der Vorhersagen später zu vereinfachen. Tupel mit Datensatz und Targetvalue werden dann wieder in dem jeweiligen Dictionary abgespeichert und ihrer RaceId zugeordnet.

In [4]:
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds',
               'driverId_1.0',
                 'driverId_2.0',
                 'driverId_3.0',
                 'driverId_4.0',
                 'driverId_5.0',
                 'driverId_8.0',
                 'driverId_10.0',
                 'driverId_13.0',
                 'driverId_15.0',
                 'driverId_16.0',
                 'driverId_17.0',
                 'driverId_18.0',
                 'driverId_20.0',
                 'driverId_22.0',
                 'driverId_24.0',
                 'driverId_30.0',
                 'driverId_37.0',
                 'driverId_39.0',
                 'driverId_67.0',
                 'driverId_153.0',
                 'driverId_154.0',
                 'driverId_155.0',
                 'driverId_807.0',
                 'driverId_808.0',
                 'driverId_811.0',
                 'driverId_812.0',
                 'driverId_813.0',
                 'driverId_814.0',
                 'driverId_815.0',
                 'driverId_816.0',
                 'driverId_817.0',
                 'driverId_818.0',
                 'driverId_819.0',
                 'driverId_820.0',
                 'driverId_821.0',
                 'driverId_822.0',
                 'driverId_823.0',
                 'driverId_824.0',
                 'driverId_825.0',
                 'driverId_826.0',
                 'driverId_827.0',
                 'driverId_828.0',
                 'driverId_829.0',
                 'driverId_830.0',
                 'driverId_831.0',
                 'driverId_832.0',
                 'driverId_833.0',
                 'driverId_834.0',
                 'driverId_835.0',
                 'driverId_836.0',
                 'driverId_837.0',
                 'driverId_838.0',
                 'driverId_839.0',
                 'driverId_840.0',
                 'driverId_841.0',
                 'driverId_842.0',
                 'driverId_843.0',
                   "status_clean"]
train, dev, test = train_dev_test(sliced_races)
train_T, dev_T, test_T = to_tensor(train, dev, test, nogo_columns)

In [5]:
print('Train:',len(train))
print('Dev:',len(dev))
print('Test:',len(test))
print('Rennen insgesamt:', len(train)+len(dev)+len(test))
print(25*'=')
print('Tensoren:')
print('Train:',len(train_T))
print('Dev:',len(dev_T))
print('Test:',len(test_T))
print('Rennen insgesamt:', len(train_T)+len(dev_T)+len(test_T))

Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137
Tensoren:
Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137


Definieren von notwendigen Klassen: 
    
Zuerst wird die Klasse für das Neuronale Netz definiert, danach die Klasse für die Optimierung der Hyperparameter Lernrate und Epochen Anzahl

In [7]:
class NetzDynamic(nn.Module):
    
    def __init__(self, layer_information):
        super(NetzDynamic,self).__init__()
        self.__layer_information = layer_information
        #definieren von einer Moduleliste, die die Layer enthalten wird
        self.layers = nn.ModuleList()
        
        #erzeugen der in layer_information übergebeben Layer und hinzufügen zu layers Moduleliste
        for specs in self.__layer_information.values():
            type_ = specs[0]
            in_ = specs[1]
            out_ = specs[2]
            if type_ == 'linear':
                self.layers.append(nn.Linear(in_,out_))
            if type_ == 'dropout':
                self.layers.append(nn.Dropout())
        
    def forward(self, x):
        #definieren eines dynamischen forward pass, activation function richtet sich nach dictionary key
        lay_idx = 0
        for activation, specs in self.__layer_information.items():
            
            if activation.startswith('relu'):
                layer = self.layers[lay_idx] #auswählen des relevanten Layers aus layers ModuleList
                if specs[0] == 'dropout':
                    x = F.relu(x.float())
                    x = layer(x)
                else:
                    x = F.relu(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('sigmoid'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.sigmoid(x.float())
                    x = layer(x)
                else:
                    x = torch.sigmoid(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('tanh'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.tanh(x.float())
                    x = layer(x)
                else:
                    x = torch.tanh(x.float())
                    x = layer(x.float())
                    
            #if activation.startswith('last'):
            #    layer = self.layers[lay_idx]
            #    if specs[0] == 'dropout':
            #        x = layer(x)
            #    else:
            #        x = layer(x.float()) 
            if activation.startswith("first"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
            if activation.startswith("no_activation"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            lay_idx += 1
        
        return x

In [8]:
class NetzDynamic_andereFunktionen(nn.Module):
    
    def __init__(self, layer_information):
        super(NetzDynamic,self).__init__()
        self.__layer_information = layer_information
        #definieren von einer Moduleliste, die die Layer enthalten wird
        self.layers = nn.ModuleList()
        
        #erzeugen der in layer_information übergebeben Layer und hinzufügen zu layers Moduleliste
        for specs in self.__layer_information.values():
            type_ = specs[0]
            in_ = specs[1]
            out_ = specs[2]
            if type_ == 'linear':
                self.layers.append(nn.Linear(in_,out_))
            if type_ == 'dropout':
                self.layers.append(nn.Dropout())
        
    def forward(self, x):
        #definieren eines dynamischen forward pass, activation function richtet sich nach dictionary key
        lay_idx = 0
        for activation, specs in self.__layer_information.items():
            
            if activation.startswith('relu'):
                layer = self.layers[lay_idx] #auswählen des relevanten Layers aus layers ModuleList
                if specs[0] == 'dropout':
                    x = F.relu(x.float())
                    x = layer(x)
                else:
                    x = F.relu(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('sigmoid'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.sigmoid(x.float())
                    x = layer(x)
                else:
                    x = torch.sigmoid(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('tanh'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.tanh(x.float())
                    x = layer(x)
                else:
                    x = torch.tanh(x.float())
                    x = layer(x.float())
                    
            #if activation.startswith('last'):
            #    layer = self.layers[lay_idx]
            #    if specs[0] == 'dropout':
            #        x = layer(x)
            #    else:
            #        x = layer(x.float()) 
            if activation.startswith("first"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
            if activation.startswith("no_activation"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            lay_idx += 1
        
        return x

In [9]:
class NetzDynamic_falsch(nn.Module):
    
    def __init__(self, layer_information):
        super(NetzDynamic,self).__init__()
        self.__layer_information = layer_information
        #definieren von einer Moduleliste, die die Layer enthalten wird
        self.layers = nn.ModuleList()
        
        #erzeugen der in layer_information übergebeben Layer und hinzufügen zu layers Moduleliste
        for specs in self.__layer_information.values():
            type_ = specs[0]
            in_ = specs[1]
            out_ = specs[2]
            if type_ == 'linear':
                self.layers.append(nn.Linear(in_,out_))
            if type_ == 'dropout':
                self.layers.append(nn.Dropout())
        
    def forward(self, x):
        #definieren eines dynamischen forward pass, activation function richtet sich nach dictionary key
        lay_idx = 0
        for activation, specs in self.__layer_information.items():
            
            if activation.startswith('relu'):
                layer = self.layers[lay_idx] #auswählen des relevanten Layers aus layers ModuleList
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
                    x = F.relu(x.float())
            if activation.startswith('sigmoid'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
                    x = torch.sigmoid(x.float())
            if activation.startswith('tanh'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
                    x = torch.tanh(x.float())
            if activation.startswith('last'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            if activation.startswith("first"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
            if activation.startswith("no_activation"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            lay_idx += 1
        
        return x

relu linear dropout linear --> in linear layer nach dropout keine activation!!!

**Baustelle**:
Es werden Netze zufällig erzeugt, mit jeweils einer festen Anzahl von Layern. Von diesen werden Variationen erzeugt. Ziel ist es zu sagen "alle Netze mit 5 Layern waren besser als alle Netze mit nur 4 Layern", um somit ein besseres Gefühl dafür zu bekommen, welche Layeranzahl am besten funktioniert. Außerdem werden die Netze mit einheitlichen Activation Funktionen erzeugt, sie haben also entweder nur ReLu, nur Tanh oder nur Sigmoid als Aktivierungsfunktion. Diese Netze sind direkt miteinander vergleichbar. Alle Netze haben ein Dropout Layer als drittes Layer.

In [117]:
dropout = 1
drop_layer = 3
start_val = 10
out_val = 1
activations = ["relu", "sigmoid", "tanh"]
neuron_count = (30,200)
variations = 4
all_netz = []
x = 0
for layer in range(7,11):
    c = 0
    while c < variations:
        netz = {}
        for l in range(layer):
            x+=1
            specs_list = []
            if l == 0:
                in_ = start_val
                out_ = random.randint(neuron_count[0],neuron_count[1])
                key = "first"
                layer_type = "linear"
                specs_list.append(layer_type)
                specs_list.append(in_)
                specs_list.append(out_)
            elif l == drop_layer-1:
                key = "no_activation"+str(x)
                layer_type = "dropout"
                layer_before = netz[list(netz.keys())[l-2]]
                out_ = netz[list(netz.keys())[l-2]][2]
                specs_list.append(layer_type)
                specs_list.append(in_)
                specs_list.append(out_)
            elif l == drop_layer:
                key = "no_activation"+str(x)
                layer_type = "linear"
                layer_before = netz[list(netz.keys())[l-2]]
                in_ = netz[list(netz.keys())[l-2]][2]
                out_ = random.randint(neuron_count[0],neuron_count[1])
                specs_list.append(layer_type)
                specs_list.append(in_)
                specs_list.append(out_)
            elif l == layer -1:
                in_ = netz[list(netz.keys())[l-1]][2]
                out_ = out_val
                key = "last"
                layer_type = "linear"
                specs_list.append(layer_type)
                specs_list.append(in_)
                specs_list.append(out_)
            else:
                key = "relu"+str(x)
                layer_type = "linear"
                layer_before = netz[list(netz.keys())[l-1]]
                in_ = netz[list(netz.keys())[l-1]][2]
                out_ = random.randint(neuron_count[0],neuron_count[1])
                specs_list.append(layer_type)
                specs_list.append(in_)
                specs_list.append(out_)
                
            netz[key]=specs_list   
        all_netz.append(netz)
        c += 1

In [118]:
#netze werden kopiert, nur die activation functions werden verändert
temp_alle_netze = []
for netz in all_netz:
    x = 0
    for activation in ["sigmoid", "tanh"]:
        netz_ = {}
        for acts, layer_specs in netz.items():
            if acts in ["first", "last"] or acts.startswith("no_activation"):
                netz_[acts] = layer_specs
            else:
                key = activation+str(x)
                netz_[key] = layer_specs
                x +=1
        temp_alle_netze.append(netz_)
all_netz = all_netz+temp_alle_netze

In [43]:
netze = {}
infos = []
max_epochs = 6
r = 0
for instruction in all_netz:
    #Netz_D = NetzDynamic(instruction)
    k = str(len(instruction.keys()))+'_l_'+list(instruction.keys())[1]+str(r)
    r+=1
    #print(k)
    netze[k] = instruction

In [10]:
class HP_Layer_Optimizer(object):
    
    def __init__ (self, 
                  layer_range = (2,8), #range mindestanzahl layer, max anzahl layer
                  input_start = 53, #anzahl inputneuronen für das Inputlayer
                  output_last = 1, #anzahl outputneuronen für das letzte layer
                  in_out_range = (30,200), #range in der sich die anzahl der Neuronen für die Hiddenlayer bewegen soll
                  random_activation = False,#boolean, die angibt ob activation functions random gepicked werden sollen
                  types = ['linear', 'dropout'],#layertypen, von denen gewählt wird
                  activations = ['relu', 'sigmoid', 'tanh'],#activation functions
                  activation_dist = [0.5,0.25,0.25],#wenn random_activation == False wird hier die Häufigkeit für jede Activation übergeben
                  dropout_num = 1,#anzahl von dropoutlayern pro NN
                  cuda = True,#soll mit cuda gearbeitet werden?
                  lr = 0.0001,#lernrate für adam optimizer
                  max_epochs = 5,#anzahl der Trainingsepochen
                  create_combinations = True, #sollen von der Klasse Netze erzeugt werden, oder sollen diese von außen in die Klasse gegeben werden?
                  create_variations = False,#soll von einer Layeranzahl mehr als ein Netz erzeugt werden?
                  num_variations = 4 #wenn Variationen erzeugt werden sollen wird eine festgelegte Zahl Netze mit 4 Layer, eine festgelegte Anzahl mit 5 Layern usw. erzeugt
                  ):
        
        self.__model = None #model, welches trainiert und getestet wird, wird hier zwischengespeichert
        self.__layer_range = layer_range #min und max anzahl von layern in Tupelform
        self.__input_start = input_start#Inputgröße für Startlayer/Inputlayer
        self.__output_last = output_last#Outputgröße für letztes Layer / Outputlayer
        self.__in_out_range = in_out_range#range in der sich die Anzahl der In- und Outputs für die Hiddenlayer bewegen soll (Tupelform)
        self.__layer_types = types#liste welche Layertypen enthält (bspw. linear oder dropout)
        self.__dropout_number = dropout_num#anzahl von layern, die pro Netz ein dropout Layer sein sollen
        self.__random_activation = random_activation#Boolean, ob die activation zufällig gewählt werden soll
        self.__create_variations = create_variations
        self.__number_of_variations = num_variations
        
        self.__combination_results = {}#dictionary, die dem schlüssel zu einer NN Kombi einen MAE zuordnet
        self.model_specs_combinations = {}#dictionary, welches die jeweiligen NN Kombinationen enthält (einem Schlüssel zugeordnet)
        self.train_data = None#trainingsdaten, die dem Optimizer übergeben werden für die Modelle
        self.test_data = None#developmentdaten, um die Modelle zu testen und Aussagen über die besten Kombinationen zu treffen
        self.lr = lr#lernrate für Modell OPtimizer (default Adam)
        self.max_epochs = max_epochs#Anzahl der Trainingsepochen
        self.cuda = cuda#Boolean, ob mit cuda gearbeitet werden soll oder nicht
        self.opt_combination = {}#dictionary enthält optimale kombination aus möglichen kombinationen
        
        self.__activations = None#entweder Liste (wenn activations random ausgewählt werden sollen), oder dictionary, 
        #wenn activation functions mit einer bestimmten Häufigkeit verwendet werden sollen
        if self.__random_activation:#random pick von activation functions
            self.__activations = activations
        else:#activation function sollen mit einer gewissen häufigkeit ausgewählt werden
            k = {}
            for a in range(len(types)):#zuordnen einer häufihkeit aus activation_dist liste zu jeder activation function aus types
                act = types[a]
                dist = activation_dist[a]
                k[act] = dist
            self.__activations = k
            
        #aufrufen der Funktion, die anhand der übergebenen Parameter Modellkombinationen erzeugt
        if create_combinations:
            self.__create_combinations()
        else:
            print('NN Kombinationen müssen in Dictionary Form selbst übergeben werden')
        
    def __train(self, epoch, optimizer):
        '''
            funktion übernimmt das Training von dem in self.__model
            zwischengespeicherten NN. 
            Epoch: Jetzige Epoche in der trainiert wird (für coolen print Befehl wichtig)
            Optimizer: Optimizer mit dem Parameter des NN aus self.__model optimiert werden
        '''
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)#.cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
            
    def __test(self):
        '''
            Funktion, die das Testen des Models aus self.__model auf den übergebenen Dev-Daten 
            (self.test_data) übernimmt und den gesamt MAE für das jeweilige Modell berechnet
        '''
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    #files.listdir(path)
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    #files.listdir(path)
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    out = self.__model(data)#.cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    #target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
    
    def get_all_information(self):
        
        print('All Model Combinations with encoding:\n', self.model_specs_combinations)
        print('Model Results:\n', self.__combination_results)
        print('Optimale Kombination:\n', self.opt_combination)
        
    def validate_combinations(self):
        
        for key, combination in self.model_specs_combinations.items():
            print(combination)
            self.__model = NetzDynamic(combination)
            optimizer = optim.Adam(self.__model.parameters(), lr = self.lr)
            #trainieren des modells
            for epoch in range(1,self.max_epochs):
                self.__train(epoch, optimizer)  
            
            result = self.__test()
            A = result.prediction.tolist()
            y = result.target.tolist()
            mae = MAE(A,y)
            
            
            
            self.__combination_results[key] = mae
            
        #finden der besten kombination nach minimalstem Error (MAE)
        key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
        best_combination = self.model_specs_combinations[key_min]
        best_combination['mae'] = self.__combination_results[key_min]
        self.opt_combination = best_combination
        #self.__combination_overview[key] = specifics
        
        
    def __create_combinations(self):
        '''
            Funktion erzeugt Dictionarys, mit NN Modellspezifikationen, die
            später gegeneinander getestet werden sollen. Kombinationen werden
            in dem Dictionary self.model_specs_combinations unter einem 
            Schlüssel abgespeichert
        '''
        
        if self.__create_variations:
            '''
            es werden mehr als ein Netz von einer bestimmten Layeranzahl erzeugt:
            wenn die Layeranzahl Range zwischen 7 und 9 liegt, self.__create_variations = True ist und
            die Variable self.__number_of_variations = 4 ist werden 4 Netze mit 7 Layern und 4 Netze
            mit 8 Layern erzeugt. Diese können dann gegeneinander getestet werden und geben eine bessere
            Übersicht über eine gute Layeranzahl.
            '''
            min_layer = self.__layer_range[0]
            max_layer = self.__layer_range[1]
            if min_layer == max_layer:
                max_layer += 1
            for layer in range(min_layer, max_layer): 
                variation_counter = 0
                while variation_counter < self.__number_of_variations:
                    variation_counter +=1 
                    dropout_counter = 0
                    act_count = 0
                    specs_dict = {}
                    middle = layer//2
                    for l in range(layer):
                        layer_specs = []
                        if self.__random_activation:#random activation pick ist aktiviert
                            act = random.choice(self.__activations)
                            key = act+str(act_count)
                            act_count += 1
                            if dropout_counter == self.__dropout_number:
                                #es wurden schon ausreichend dropout layer erzeugt
                                l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                            else:
                                l_ = random.choice(self.__layer_types)
                                if l_ == 'dropout':
                                    dropout_counter +=1
                                    
                            if l == 0:
                                in_ = self.__input_start
                                if l_ == 'dropout':
                                    dropout_counter = dropout_counter-1
                                    l_ = 'linear'
                                range_start = self.__in_out_range[0]
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            else:
                                layer_before = specs_dict[list(specs_dict.keys())[-1]]
                                out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                                l_type = layer_before[0]
                                i_ltype = -2
                                while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                    #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                    layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                    l_type = layer_before[0]
                                    out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                    i_ltype = i_ltype -1
                                if l <= middle: #in der ertsen hälfte nimmt output zu
                                    in_ = out_alt
                                    range_start = out_alt
                                    range_end = self.__in_out_range[1]
                                    out_ = random.randint(range_start, range_end)
                                else:#in zweiter hälfte der layer nimmt output wieder ab
                                    in_ = out_alt
                                    range_start = self.__in_out_range[0]
                                    range_end = out_alt
                                    out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            if l == layer-1:
                                l_ = 'linear'
                                layer_specs = [l_,in_,self.__output_last]
                                specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                                
                            elif l == 0:
                                specs_dict['first'] = layer_specs
                            else:
                                #if l_ == 'dropout':#keine activation function bei dropout layern
                                #    key = 'no_activation'+str(act_count)
                                if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                    key = 'no_activation'+str(act_count)
                                specs_dict[key] = layer_specs
                    key = random.randint(0,10000)
                    while key in list(self.model_specs_combinations.keys()):
                         key = random.randint(0,10000)
                    self.model_specs_combinations[key]= specs_dict   
                    print(specs_dict,'\n')
                
        else:
            min_layer = self.__layer_range[0]
            max_layer = self.__layer_range[1]
            if min_layer == max_layer:
                max_layer += 1
            for layer in range(min_layer, max_layer): 
                
                dropout_counter = 0
                act_count = 0
                specs_dict = {}
                middle = layer//2
                for l in range(layer):
                    layer_specs = []
                    if self.__random_activation:#random activation pick ist aktiviert
                        act = random.choice(self.__activations)
                        key = act+str(act_count)
                        act_count += 1
                        if dropout_counter == self.__dropout_number:
                            #es wurden schon ausreichend dropout layer erzeugt
                            l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                        else:
                            l_ = random.choice(self.__layer_types)
                            if l_ == 'dropout':
                                dropout_counter +=1
                                
                        if l == 0:
                            in_ = self.__input_start
                            if l_ == 'dropout':
                                dropout_counter = dropout_counter-1
                                l_ = 'linear'
                            range_start = self.__in_out_range[0]
                            range_end = self.__in_out_range[1]
                            out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        else:
                            layer_before = specs_dict[list(specs_dict.keys())[-1]]
                            out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                            l_type = layer_before[0]
                            i_ltype = -2
                            while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                l_type = layer_before[0]
                                out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                i_ltype = i_ltype -1
                            if l <= middle: #in der ertsen hälfte nimmt output zu
                                in_ = out_alt
                                range_start = out_alt
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                            else:#in zweiter hälfte der layer nimmt output wieder ab
                                in_ = out_alt
                                range_start = self.__in_out_range[0]
                                range_end = out_alt
                                out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        #if l == layer-1:
                        #    l_ = 'linear'
                        #    layer_specs = [l_,in_,self.__output_last]
                        #    specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                            
                        #elif l == 0:
                        if l == 0:
                            specs_dict['first'] = layer_specs
                        else:
                            #if l_ == 'dropout':#keine activation function bei dropout layern
                            #    key = 'no_activation'+str(act_count)
                            if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                key = 'no_activation'+str(act_count)
                            specs_dict[key] = layer_specs
                key = random.randint(0,10000)
                while key in list(self.model_specs_combinations.keys()):
                     key = random.randint(0,10000)
                self.model_specs_combinations[key]= specs_dict   
                print(specs_dict)
            

In [11]:
l = HP_Layer_Optimizer(layer_range=(8,12),
                       max_epochs = 6,
                       input_start = 10,
                       activations = ['relu'], 
                       random_activation=True,
                       create_combinations=True, 
                       create_variations = True, 
                       cuda = False)
#l.model_specs_combinations = netze
l.train_data = train_T
l.test_data = dev_T
l.validate_combinations()
l.get_all_information()
opt_combination = l.opt_combination

{'first': ['linear', 10, 77], 'relu1': ['linear', 77, 143], 'relu2': ['dropout', 143, 170], 'no_activation4': ['linear', 143, 181], 'relu4': ['linear', 181, 196], 'relu5': ['linear', 196, 95], 'relu6': ['linear', 95, 73], 'last': ['linear', 73, 1]} 

{'first': ['linear', 10, 42], 'relu1': ['dropout', 42, 180], 'no_activation3': ['linear', 42, 81], 'relu3': ['linear', 81, 192], 'relu4': ['linear', 192, 196], 'relu5': ['linear', 196, 142], 'relu6': ['linear', 142, 96], 'last': ['linear', 96, 1]} 

{'first': ['linear', 10, 169], 'relu1': ['dropout', 169, 182], 'no_activation3': ['linear', 169, 173], 'relu3': ['linear', 173, 191], 'relu4': ['linear', 191, 197], 'relu5': ['linear', 197, 143], 'relu6': ['linear', 143, 79], 'last': ['linear', 79, 1]} 

{'first': ['linear', 10, 191], 'relu1': ['linear', 191, 196], 'relu2': ['dropout', 196, 200], 'no_activation4': ['linear', 196, 198], 'relu4': ['linear', 198, 200], 'relu5': ['linear', 200, 61], 'relu6': ['linear', 61, 43], 'last': ['linear', 4

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 73])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Train Epoche: 1 [10/96 (10%)]	Loss: 62.772888
Train Epoche: 1 [11/96 (11%)]	Loss: 34.767738
Train Epoche: 1 [12/96 (12%)]	Loss: 396.239532
Train Epoche: 1 [13/96 (14%)]	Loss: 15.370191
Train Epoche: 1 [14/96 (15%)]	Loss: 221.052856
Train Epoche: 1 [15/96 (16%)]	Loss: 395.544342
Train Epoche: 1 [16/96 (17%)]	Loss: 0.825584
Train Epoche: 1 [17/96 (18%)]	Loss: 22.772278
Train Epoche: 1 [18/96 (19%)]	Loss: 184.733765
Train Epoche: 1 [19/96 (20%)]	Loss: 240.234512
Train Epoche: 1 [20/96 (21%)]	Loss: 62.465485
Train Epoche: 1 [21/96 (22%)]	Loss: 320.430450
Train Epoche: 1 [22/96 (23%)]	Loss: 480.141479
Train Epoche: 1 [23/96 (24%)]	Loss: 570.207275
Train Epoche: 1 [24/96 (25%)]	Loss: 221.194809
Train Epoche: 1 [25/96 (26%)]	Loss: 193.301254
Train Epoche: 1 [26/96 (27%)]	Loss: 47.421867
Train Epoche: 1 [27/96 (28%)]	Loss: 78.733643
Train Epoche: 1 [28/96 (29%)]	Loss: 139.864731
Train Epoche: 1 [29/96 (30%)]	Loss: 34.447769
Train Epoche: 1 [30/96 (31%)]	Loss: 15.063132
Train Epoche: 1 [31/96 

Train Epoche: 1 [200/96 (208%)]	Loss: 3.236683
Train Epoche: 1 [201/96 (209%)]	Loss: 6.887524
Train Epoche: 1 [202/96 (210%)]	Loss: 62.876732
Train Epoche: 1 [203/96 (211%)]	Loss: 19.031940
Train Epoche: 1 [204/96 (212%)]	Loss: 6.489679
Train Epoche: 1 [205/96 (214%)]	Loss: 68.662766
Train Epoche: 1 [206/96 (215%)]	Loss: 90.342331
Train Epoche: 1 [207/96 (216%)]	Loss: 12.834703
Train Epoche: 1 [208/96 (217%)]	Loss: 9.900310
Train Epoche: 1 [209/96 (218%)]	Loss: 23.381529
Train Epoche: 1 [210/96 (219%)]	Loss: 60.649937
Train Epoche: 1 [211/96 (220%)]	Loss: 32.461937
Train Epoche: 1 [212/96 (221%)]	Loss: 25.472420
Train Epoche: 1 [213/96 (222%)]	Loss: 26.875669
Train Epoche: 1 [214/96 (223%)]	Loss: 97.516403
Train Epoche: 1 [215/96 (224%)]	Loss: 92.421471
Train Epoche: 1 [216/96 (225%)]	Loss: 145.300568
Train Epoche: 1 [217/96 (226%)]	Loss: 118.973099
Train Epoche: 1 [218/96 (227%)]	Loss: 13.402946
Train Epoche: 1 [219/96 (228%)]	Loss: 14.841379
Train Epoche: 1 [220/96 (229%)]	Loss: 39.5

Train Epoche: 1 [390/96 (406%)]	Loss: 39.738010
Train Epoche: 1 [391/96 (407%)]	Loss: 10.906814
Train Epoche: 1 [392/96 (408%)]	Loss: 7.234331
Train Epoche: 1 [393/96 (409%)]	Loss: 0.992789
Train Epoche: 1 [394/96 (410%)]	Loss: 183.936554
Train Epoche: 1 [395/96 (411%)]	Loss: 17.678335
Train Epoche: 1 [396/96 (412%)]	Loss: 41.678051
Train Epoche: 1 [397/96 (414%)]	Loss: 143.148697
Train Epoche: 1 [398/96 (415%)]	Loss: 93.661591
Train Epoche: 1 [399/96 (416%)]	Loss: 40.389679
Train Epoche: 1 [400/96 (417%)]	Loss: 21.447989
Train Epoche: 1 [401/96 (418%)]	Loss: 2.849973
Train Epoche: 1 [402/96 (419%)]	Loss: 2.537155
Train Epoche: 1 [403/96 (420%)]	Loss: 23.462650
Train Epoche: 1 [404/96 (421%)]	Loss: 0.755511
Train Epoche: 1 [405/96 (422%)]	Loss: 0.115567
Train Epoche: 1 [406/96 (423%)]	Loss: 5.159516
Train Epoche: 1 [407/96 (424%)]	Loss: 40.537903
Train Epoche: 1 [408/96 (425%)]	Loss: 39.596500
Train Epoche: 1 [409/96 (426%)]	Loss: 275.117096
Train Epoche: 1 [410/96 (427%)]	Loss: 207.20

...

Train Epoche: 1 [973/96 (1014%)]	Loss: 1.464396
Train Epoche: 1 [974/96 (1015%)]	Loss: 238.289673
Train Epoche: 1 [975/96 (1016%)]	Loss: 198.677612
Train Epoche: 1 [976/96 (1017%)]	Loss: 58.471233
Train Epoche: 1 [977/96 (1018%)]	Loss: 2.373745
Train Epoche: 1 [978/96 (1019%)]	Loss: 122.759781
Train Epoche: 1 [979/96 (1020%)]	Loss: 5.868766
Train Epoche: 1 [980/96 (1021%)]	Loss: 18.766680
Train Epoche: 1 [981/96 (1022%)]	Loss: 12.435719
Train Epoche: 1 [982/96 (1023%)]	Loss: 54.365021
Train Epoche: 1 [983/96 (1024%)]	Loss: 4.338570
Train Epoche: 1 [984/96 (1025%)]	Loss: 0.571979
Train Epoche: 1 [985/96 (1026%)]	Loss: 0.115165
Train Epoche: 1 [986/96 (1027%)]	Loss: 3.914530
Train Epoche: 1 [987/96 (1028%)]	Loss: 0.424001
Train Epoche: 1 [988/96 (1029%)]	Loss: 21.517200
Train Epoche: 1 [989/96 (1030%)]	Loss: 34.805454
Train Epoche: 1 [990/96 (1031%)]	Loss: 16.333323
Train Epoche: 1 [991/96 (1032%)]	Loss: 11.785390
Train Epoche: 1 [992/96 (1033%)]	Loss: 27.216831
Train Epoche: 1 [993/96 (

Train Epoche: 1 [1145/96 (1193%)]	Loss: 1.024297
Train Epoche: 1 [1146/96 (1194%)]	Loss: 36.322571
Train Epoche: 1 [1147/96 (1195%)]	Loss: 0.410565
Train Epoche: 1 [1148/96 (1196%)]	Loss: 9.524872
Train Epoche: 1 [1149/96 (1197%)]	Loss: 11.379649
Train Epoche: 1 [1150/96 (1198%)]	Loss: 32.535675
Train Epoche: 1 [1151/96 (1199%)]	Loss: 39.703999
Train Epoche: 1 [1152/96 (1200%)]	Loss: 0.032461
Train Epoche: 1 [1153/96 (1201%)]	Loss: 4.004216
Train Epoche: 1 [1154/96 (1202%)]	Loss: 23.969515
Train Epoche: 1 [1155/96 (1203%)]	Loss: 45.560780
Train Epoche: 1 [1156/96 (1204%)]	Loss: 1.179672
Train Epoche: 1 [1157/96 (1205%)]	Loss: 15.310838
Train Epoche: 1 [1158/96 (1206%)]	Loss: 36.338421
Train Epoche: 1 [1159/96 (1207%)]	Loss: 33.488377
Train Epoche: 1 [1160/96 (1208%)]	Loss: 4.967696
Train Epoche: 1 [1161/96 (1209%)]	Loss: 64.890144
Train Epoche: 1 [1162/96 (1210%)]	Loss: 17.401361
Train Epoche: 1 [1163/96 (1211%)]	Loss: 6.960855
Train Epoche: 1 [1164/96 (1212%)]	Loss: 14.731134
Train Ep

Train Epoche: 1 [1320/96 (1375%)]	Loss: 15.324812
Train Epoche: 1 [1321/96 (1376%)]	Loss: 17.992195
Train Epoche: 1 [1322/96 (1377%)]	Loss: 14.362525
Train Epoche: 1 [1323/96 (1378%)]	Loss: 212.326111
Train Epoche: 1 [1324/96 (1379%)]	Loss: 4.001437
Train Epoche: 1 [1325/96 (1380%)]	Loss: 3.203600
Train Epoche: 1 [1326/96 (1381%)]	Loss: 18.850498
Train Epoche: 1 [1327/96 (1382%)]	Loss: 186.005646
Train Epoche: 1 [1328/96 (1383%)]	Loss: 7.299314
Train Epoche: 1 [1329/96 (1384%)]	Loss: 20.561178
Train Epoche: 1 [1330/96 (1385%)]	Loss: 117.645264
Train Epoche: 1 [1331/96 (1386%)]	Loss: 0.392610
Train Epoche: 1 [1332/96 (1388%)]	Loss: 0.371951
Train Epoche: 1 [1333/96 (1389%)]	Loss: 2.502972
Train Epoche: 1 [1334/96 (1390%)]	Loss: 104.215324
Train Epoche: 1 [1335/96 (1391%)]	Loss: 28.962883
Train Epoche: 1 [1336/96 (1392%)]	Loss: 214.787018
Train Epoche: 1 [1337/96 (1393%)]	Loss: 20.584948
Train Epoche: 1 [1338/96 (1394%)]	Loss: 11.016150
Train Epoche: 1 [1339/96 (1395%)]	Loss: 13.286571
T

Train Epoche: 1 [1505/96 (1568%)]	Loss: 56.683247
Train Epoche: 1 [1506/96 (1569%)]	Loss: 161.624786
Train Epoche: 1 [1507/96 (1570%)]	Loss: 19.741293
Train Epoche: 1 [1508/96 (1571%)]	Loss: 0.227330
Train Epoche: 1 [1509/96 (1572%)]	Loss: 47.442284
Train Epoche: 1 [1510/96 (1573%)]	Loss: 70.571579
Train Epoche: 1 [1511/96 (1574%)]	Loss: 38.556194
Train Epoche: 1 [1512/96 (1575%)]	Loss: 17.014875
Train Epoche: 1 [1513/96 (1576%)]	Loss: 46.534645
Train Epoche: 1 [1514/96 (1577%)]	Loss: 10.656707
Train Epoche: 1 [1515/96 (1578%)]	Loss: 2.453399
Train Epoche: 1 [1516/96 (1579%)]	Loss: 15.673967
Train Epoche: 1 [1517/96 (1580%)]	Loss: 197.034500
Train Epoche: 1 [1518/96 (1581%)]	Loss: 2.902243
Train Epoche: 1 [1519/96 (1582%)]	Loss: 230.786514
Train Epoche: 1 [1520/96 (1583%)]	Loss: 0.286437
Train Epoche: 1 [1521/96 (1584%)]	Loss: 25.733242
Train Epoche: 1 [1522/96 (1585%)]	Loss: 77.682564
Train Epoche: 1 [1523/96 (1586%)]	Loss: 199.939331
Train Epoche: 1 [1524/96 (1588%)]	Loss: 5.089627
T

KeyboardInterrupt: 

In [124]:
#opt_layer = HP_Layer_Optimizer(layer_range=(6,10),
#                               in_out_range = (30,200),
#                               random_activation=True,#randompick von activation function, momentan geht False nicht
#                               cuda = True,
#                               dropout_num = 1,#anzahl zu erzeugender dropoutlayer
#                               lr = 0.0001,#lernrate für adam optimizer
#                               max_epochs = 5,#anzahl der Trainingsepochen
#                               activations = ['relu'],#normalerweise relu, tanh, sigmoid --> auskommentieren wenn wieder default
#                              input_start = 10
#                              )
#opt_layer.train_data = train_T
#opt_layer.test_data = dev_T
##opt_layer.validate_combinations()
##opt_layer.get_all_information()
##opt_combination = opt_layer.opt_combination

In [12]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        #self.fc1 = nn.Linear(53, 150)
        self.fc1 = nn.Linear(10, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        #x = self.dropout(x)
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x
        

In [13]:
julian = {'first':['linear', 10,150],
         'relu0':['linear',150,180],
         'relu1':['dropout',0,0],
         'no_activation0':['linear',180,190],
         'relu2':['linear',190,120],
         'relu3':['linear',120,100],
         'relu4':['linear',100,70],
         'relu5':['linear',70,30],
         'relu6':['linear',30,1]}

In [14]:
class HP_Optimizer(object):
    
    def __init__(self, lr_range = (0.0001,0.0001), step_size = 0.0001, max_epochs = (2,2), opt = 'Adam', cuda = True, dynamic = False, dyn_combination = {}):
        
        self.__model = Netz()
        self.__lr = lr_range
        self.__epochs = max_epochs
        self.__optimizer = opt
        self.__steps = step_size
        self.__combination_results = {}
        self.__combination_overview = {}
        self.train_data = None
        self.test_data = None
        self.cuda = cuda
        self.opt_combination = {}
        self.__dynamic = dynamic
        self.__dyn_combination = dyn_combination
        
        
    def validate_combinations(self):
        
        specifics = {}
        if self.__optimizer == 'Adam':
            
            #definieren der range für die lernratenoptimierung
            '''
            
            IST HIER WAS KAPUTT?
            
            '''
            if isinstance(self.__lr, tuple):
                lr_s = self.__lr[0]
                lr_e = self.__lr[1]
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if lr_s == lr_e:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = lr_s
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in np.arange(lr_s,lr_e, self.__steps):
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if lr_s == lr_e:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = lr_s
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in np.arange(lr_s,lr_e, self.__steps):
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
            elif isinstance(self.__lr, list):
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if len(self.__lr) ==1:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = self.__lr[0]
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in self.__lr:
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if len(self.__lr) == 1:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = self.__lr[0]
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in self.__lr:
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
                
        else:
            raise ('No valid optimizer given! Try Adam for example!')
            
    def __train(self, epoch, optimizer):
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)#.cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    #print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    #    epoch, batch_id *len(data), len(train_data),
                    #100. * batch_id / len(train_data), loss.item()))
                    batch_id +=1
                    
            
    def __test(self):
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    #files.listdir(path)
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    #files.listdir(path)
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    out = self.__model(data)#.cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    #target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
         
            
    def get_all_information(self):
        
        print('Chosen Model:',self.__model)
        print('Learningrate Range:',self.__lr)
        print('Maximum Epochs:', self.__epochs)
        print('Chosen Optimizer:', self.__optimizer)
        print('Result Encoding:', self.__combination_overview)
        print('Results:', self.__combination_results)
        print('Optimale Kombination:', self.opt_combination)
        
    def help(self):
        print('Parameters with defaults:\nlr_range --> (0.0001,0.0001),\nstep_size--> 0.0001,\nmax_epochs-->(2,2),\nopt-->"Adam",\ncuda=True')
        print('lr_range: Tupel with learnrate range')
        print('step_size: float/int for step_size of learnrate')
        print('max_epochs: Tupel with number of epochs range')
        print('opt: Optimizer (by default Adam)')
        print('cuda: True/False if cuda should be used, default = True\n')
        print('Attributes:')
        print('set self.train_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('set self.test_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('self.opt_combination: Dictionary which contains the best combination of the given parameters\n')
        print('Methods:')
        print('call self.validate_combination() to compare all combinations')
        print('get all information/results with self.get_all_information()')

Paralleles Aufrufen des Optimierers, übergeben der zuvor erzeugten Test- und Trainingsdatensätze, angeben der Intervalle für die die Hyperparameter getestet werden sollen (Lernrate und Epochenanzahl). Für das dynamische Modell wird das Dictionary übergeben, welches von dem Layer Optimizer am besten bewertet wurde. Der "normale" HP_Optimizer basiert auf der Klasse Netz().

In [16]:
h = HP_Optimizer(lr_range = [0.0002,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False)
opt_={}
#for key in opt_combination.keys():
#    if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
#        opt_[key] = opt_combination[key]
#h_dynamic = HP_Optimizer(lr_range = [0.0002,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False, dynamic = True, dyn_combination = opt_)
h_dynamic = HP_Optimizer(lr_range = [0.0002,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False, dynamic = True, dyn_combination = julian)
h.help()
h.train_data = train_T
h_dynamic.train_data = train_T

h.test_data = dev_T
h_dynamic.test_data = dev_T

h.validate_combinations()
h_dynamic.validate_combinations()

h.get_all_information()
h_dynamic.get_all_information()

Parameters with defaults:
lr_range --> (0.0001,0.0001),
step_size--> 0.0001,
max_epochs-->(2,2),
opt-->"Adam",
cuda=True
lr_range: Tupel with learnrate range
step_size: float/int for step_size of learnrate
max_epochs: Tupel with number of epochs range
opt: Optimizer (by default Adam)
cuda: True/False if cuda should be used, default = True

Attributes:
set self.train_data as dictionary with races (form: {raceId: race(dataframe)})
set self.test_data as dictionary with races (form: {raceId: race(dataframe)})
self.opt_combination: Dictionary which contains the best combination of the given parameters

Methods:
call self.validate_combination() to compare all combinations
get all information/results with self.get_all_information()


c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

Chosen Model: Netz(
  (fc1): Linear(in_features=10, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=180, bias=True)
  (fc3): Linear(in_features=180, out_features=190, bias=True)
  (fc4): Linear(in_features=190, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=70, bias=True)
  (fc7): Linear(in_features=70, out_features=30, bias=True)
  (fc8): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Learningrate Range: [0.0002, 0.00045]
Maximum Epochs: (3, 5)
Chosen Optimizer: Adam
Result Encoding: {4645: {'lr': 0.0002, 'epochen': 3}, 6904: {'lr': 0.00045, 'epochen': 3}, 5686: {'lr': 0.0002, 'epochen': 4}, 3013: {'lr': 0.00045, 'epochen': 4, 'mae': 2.2250489236790605}}
Results: {4645: 2.5538160469667317, 6904: 2.3424657534246576, 5686: 2.2915851272015657, 3013: 2.2250489236790605}
Optimale Kombination: {'lr': 0.00045, 'epochen': 4, 'mae'

Es werden die als optimal in den gegebenen Intervallen genommenen Werte aus dem Optimierer genommen und ein neues Modell mit diesen Parametern trainiert. Zum Schluss werden die Ergebnisse des Testlaufes auf den Testdaten ausgegeben.
Dies geschieht genauso mit dem zuvor dynamisch erzeugten Modell. Diese können so direkt miteinander verglichen werden.

In [22]:
max_epochs = h.opt_combination['epochen']
max_epochs_dyn = h_dynamic.opt_combination['epochen']
lr = h.opt_combination['lr']
lr_dynamic = h_dynamic.opt_combination['lr']

#cuda = input('Cuda? [y/n]: ')
opt_combination = julian
cuda = 'n'
model = Netz()
opt={}
for key in opt_combination.keys():
#for key in julian.keys():
    if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
        opt[key] = opt_combination[key]
#model_dynamic = NetzDynamic(opt)
model_dynamic = NetzDynamic(julian)
if cuda.lower() == 'y':
    model.cuda() 
    model_dynamic.cuda()

    
optimizer = optim.Adam(model.parameters(), lr = lr)     
optimizer_d = optim.Adam(model_dynamic.parameters(), lr = lr_dynamic)   

def train_cuda(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
        
    #random.shuffle(train_data)
def train_(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)#.cuda()
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
            
def train_cuda_dynamic(epoch):
    model_dynamic.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer.zero_grad()
            out = model_dynamic(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
        
    #random.shuffle(train_data)
def train_dynamic(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)#.cuda()
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            optimizer.zero_grad()
            out = model_dynamic(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
        
    #random.shuffle(train_data)
if cuda.lower() == 'y':
    for epoch in range(1,max_epochs):
        train_cuda(epoch) 
    for epoch in range(1,max_epochs_dyn):
        train_cuda_dynamic(epoch) 
else:
    for epoch in range(1,max_epochs):
        train_(epoch)  
    for epoch in range(1,max_epochs_dyn):
        train_dynamic(epoch)  
        

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Train Epoche: 1 [0/96 (0%)]	Loss: 395.346191
Train Epoche: 1 [1/96 (1%)]	Loss: 140.219345
Train Epoche: 1 [2/96 (2%)]	Loss: 78.740875
Train Epoche: 1 [3/96 (3%)]	Loss: 116.809723
Train Epoche: 1 [4/96 (4%)]	Loss: 391.713287
Train Epoche: 1 [5/96 (5%)]	Loss: 45.097420
Train Epoche: 1 [6/96 (6%)]	Loss: 95.557068
Train Epoche: 1 [7/96 (7%)]	Loss: 162.535187
Train Epoche: 1 [8/96 (8%)]	Loss: 7.424778
Train Epoche: 1 [9/96 (9%)]	Loss: 2.785810
Train Epoche: 1 [10/96 (10%)]	Loss: 58.264370
Train Epoche: 1 [11/96 (11%)]	Loss: 30.966530
Train Epoche: 1 [12/96 (12%)]	Loss: 384.086670
Train Epoche: 1 [13/96 (14%)]	Loss: 12.478821
Train Epoche: 1 [14/96 (15%)]	Loss: 208.961334
Train Epoche: 1 [15/96 (16%)]	Loss: 375.262543
Train Epoche: 1 [16/96 (17%)]	Loss: 0.015841
Train Epoche: 1 [17/96 (18%)]	Loss: 15.986426
Train Epoche: 1 [18/96 (19%)]	Loss: 155.796310
Train Epoche: 1 [19/96 (20%)]	Loss: 211.171753
Train Epoche: 1 [20/96 (21%)]	Loss: 52.303596
Train Epoche: 1 [21/96 (22%)]	Loss: 296.412506


Train Epoche: 1 [190/96 (198%)]	Loss: 0.051629
Train Epoche: 1 [191/96 (199%)]	Loss: 10.067822
Train Epoche: 1 [192/96 (200%)]	Loss: 0.315764
Train Epoche: 1 [193/96 (201%)]	Loss: 30.583750
Train Epoche: 1 [194/96 (202%)]	Loss: 0.034433
Train Epoche: 1 [195/96 (203%)]	Loss: 0.443062
Train Epoche: 1 [196/96 (204%)]	Loss: 161.874374
Train Epoche: 1 [197/96 (205%)]	Loss: 1.814044
Train Epoche: 1 [198/96 (206%)]	Loss: 2.101446
Train Epoche: 1 [199/96 (207%)]	Loss: 10.311118
Train Epoche: 1 [200/96 (208%)]	Loss: 3.314520
Train Epoche: 1 [201/96 (209%)]	Loss: 15.604074
Train Epoche: 1 [202/96 (210%)]	Loss: 32.997906
Train Epoche: 1 [203/96 (211%)]	Loss: 0.422692
Train Epoche: 1 [204/96 (212%)]	Loss: 0.033852
Train Epoche: 1 [205/96 (214%)]	Loss: 49.407192
Train Epoche: 1 [206/96 (215%)]	Loss: 126.914436
Train Epoche: 1 [207/96 (216%)]	Loss: 0.612019
Train Epoche: 1 [208/96 (217%)]	Loss: 1.827232
Train Epoche: 1 [209/96 (218%)]	Loss: 10.872366
Train Epoche: 1 [210/96 (219%)]	Loss: 69.011467
T

Train Epoche: 1 [385/96 (401%)]	Loss: 13.385541
Train Epoche: 1 [386/96 (402%)]	Loss: 1.008066
Train Epoche: 1 [387/96 (403%)]	Loss: 234.450134
Train Epoche: 1 [388/96 (404%)]	Loss: 10.620797
Train Epoche: 1 [389/96 (405%)]	Loss: 11.427537
Train Epoche: 1 [390/96 (406%)]	Loss: 34.632710
Train Epoche: 1 [391/96 (407%)]	Loss: 8.281985
Train Epoche: 1 [392/96 (408%)]	Loss: 9.300157
Train Epoche: 1 [393/96 (409%)]	Loss: 0.424898
Train Epoche: 1 [394/96 (410%)]	Loss: 155.079056
Train Epoche: 1 [395/96 (411%)]	Loss: 10.281392
Train Epoche: 1 [396/96 (412%)]	Loss: 6.639551
Train Epoche: 1 [397/96 (414%)]	Loss: 448.911926
Train Epoche: 1 [398/96 (415%)]	Loss: 0.036248
Train Epoche: 1 [399/96 (416%)]	Loss: 35.738178
Train Epoche: 1 [400/96 (417%)]	Loss: 46.793823
Train Epoche: 1 [401/96 (418%)]	Loss: 0.064482
Train Epoche: 1 [402/96 (419%)]	Loss: 4.259655
Train Epoche: 1 [403/96 (420%)]	Loss: 11.213974
Train Epoche: 1 [404/96 (421%)]	Loss: 0.352768
Train Epoche: 1 [405/96 (422%)]	Loss: 0.097441

Train Epoche: 1 [579/96 (603%)]	Loss: 8.911937
Train Epoche: 1 [580/96 (604%)]	Loss: 8.544115
Train Epoche: 1 [581/96 (605%)]	Loss: 53.269104
Train Epoche: 1 [582/96 (606%)]	Loss: 29.108273
Train Epoche: 1 [583/96 (607%)]	Loss: 0.538212
Train Epoche: 1 [584/96 (608%)]	Loss: 0.013294
Train Epoche: 1 [585/96 (609%)]	Loss: 6.541989
Train Epoche: 1 [586/96 (610%)]	Loss: 33.568779
Train Epoche: 1 [587/96 (611%)]	Loss: 0.810262
Train Epoche: 1 [588/96 (612%)]	Loss: 7.711075
Train Epoche: 1 [589/96 (614%)]	Loss: 14.492742
Train Epoche: 1 [590/96 (615%)]	Loss: 11.858143
Train Epoche: 1 [591/96 (616%)]	Loss: 0.201088
Train Epoche: 1 [592/96 (617%)]	Loss: 27.353403
Train Epoche: 1 [593/96 (618%)]	Loss: 0.216259
Train Epoche: 1 [594/96 (619%)]	Loss: 53.214062
Train Epoche: 1 [595/96 (620%)]	Loss: 20.328749
Train Epoche: 1 [596/96 (621%)]	Loss: 38.221157
Train Epoche: 1 [597/96 (622%)]	Loss: 42.589100
Train Epoche: 1 [598/96 (623%)]	Loss: 0.049265
Train Epoche: 1 [599/96 (624%)]	Loss: 46.621479
Tr

...

Train Epoche: 2 [1300/96 (1354%)]	Loss: 287.617828
Train Epoche: 2 [1301/96 (1355%)]	Loss: 25.256376
Train Epoche: 2 [1302/96 (1356%)]	Loss: 3.915358
Train Epoche: 2 [1303/96 (1357%)]	Loss: 224.345642
Train Epoche: 2 [1304/96 (1358%)]	Loss: 167.790222
Train Epoche: 2 [1305/96 (1359%)]	Loss: 0.935785
Train Epoche: 2 [1306/96 (1360%)]	Loss: 15.810918
Train Epoche: 2 [1307/96 (1361%)]	Loss: 99.282471
Train Epoche: 2 [1308/96 (1362%)]	Loss: 255.569763
Train Epoche: 2 [1309/96 (1364%)]	Loss: 50.056831
Train Epoche: 2 [1310/96 (1365%)]	Loss: 122.135429
Train Epoche: 2 [1311/96 (1366%)]	Loss: 143.230713
Train Epoche: 2 [1312/96 (1367%)]	Loss: 399.369232
Train Epoche: 2 [1313/96 (1368%)]	Loss: 167.752029
Train Epoche: 2 [1314/96 (1369%)]	Loss: 255.409851
Train Epoche: 2 [1315/96 (1370%)]	Loss: 35.652557
Train Epoche: 2 [1316/96 (1371%)]	Loss: 223.787567
Train Epoche: 2 [1317/96 (1372%)]	Loss: 80.398216
Train Epoche: 2 [1318/96 (1373%)]	Loss: 48.586681
Train Epoche: 2 [1319/96 (1374%)]	Loss: 3.

Train Epoche: 2 [1472/96 (1533%)]	Loss: 3.817421
Train Epoche: 2 [1473/96 (1534%)]	Loss: 48.807827
Train Epoche: 2 [1474/96 (1535%)]	Loss: 63.362209
Train Epoche: 2 [1475/96 (1536%)]	Loss: 287.825989
Train Epoche: 2 [1476/96 (1538%)]	Loss: 254.961746
Train Epoche: 2 [1477/96 (1539%)]	Loss: 80.655380
Train Epoche: 2 [1478/96 (1540%)]	Loss: 168.191666
Train Epoche: 2 [1479/96 (1541%)]	Loss: 223.833069
Train Epoche: 2 [1480/96 (1542%)]	Loss: 142.978622
Train Epoche: 2 [1481/96 (1543%)]	Loss: 15.820403
Train Epoche: 2 [1482/96 (1544%)]	Loss: 24.521843
Train Epoche: 2 [1483/96 (1545%)]	Loss: 99.206253
Train Epoche: 2 [1484/96 (1546%)]	Loss: 119.756989
Train Epoche: 2 [1485/96 (1547%)]	Loss: 0.963506
Train Epoche: 2 [1486/96 (1548%)]	Loss: 579.165955
Train Epoche: 2 [1487/96 (1549%)]	Loss: 440.141174
Train Epoche: 2 [1488/96 (1550%)]	Loss: 483.953186
Train Epoche: 2 [1489/96 (1551%)]	Loss: 534.451965
Train Epoche: 2 [1490/96 (1552%)]	Loss: 325.642670
Train Epoche: 2 [1491/96 (1553%)]	Loss: 1

Train Epoche: 2 [1646/96 (1715%)]	Loss: 37.065769
Train Epoche: 2 [1647/96 (1716%)]	Loss: 361.182526
Train Epoche: 2 [1648/96 (1717%)]	Loss: 257.925476
Train Epoche: 2 [1649/96 (1718%)]	Loss: 577.424805
Train Epoche: 2 [1650/96 (1719%)]	Loss: 574.202148
Train Epoche: 2 [1651/96 (1720%)]	Loss: 3.872738
Train Epoche: 2 [1652/96 (1721%)]	Loss: 288.242737
Train Epoche: 2 [1653/96 (1722%)]	Loss: 223.992935
Train Epoche: 2 [1654/96 (1723%)]	Loss: 168.424759
Train Epoche: 2 [1655/96 (1724%)]	Loss: 195.167755
Train Epoche: 2 [1656/96 (1725%)]	Loss: 0.943093
Train Epoche: 2 [1657/96 (1726%)]	Loss: 143.102676
Train Epoche: 2 [1658/96 (1727%)]	Loss: 80.096199
Train Epoche: 2 [1659/96 (1728%)]	Loss: 120.068382
Train Epoche: 2 [1660/96 (1729%)]	Loss: 63.411568
Train Epoche: 2 [1661/96 (1730%)]	Loss: 24.527525
Train Epoche: 2 [1662/96 (1731%)]	Loss: 15.771791
Train Epoche: 2 [1663/96 (1732%)]	Loss: 35.414433
Train Epoche: 2 [1664/96 (1733%)]	Loss: 99.539734
Train Epoche: 2 [1665/96 (1734%)]	Loss: 49

Train Epoche: 2 [1842/96 (1919%)]	Loss: 15.672968
Train Epoche: 2 [1843/96 (1920%)]	Loss: 63.322708
Train Epoche: 2 [1844/96 (1921%)]	Loss: 80.393616
Train Epoche: 2 [1845/96 (1922%)]	Loss: 48.679543
Train Epoche: 2 [1846/96 (1923%)]	Loss: 3.896017
Train Epoche: 2 [1847/96 (1924%)]	Loss: 8.774626
Train Epoche: 2 [1848/96 (1925%)]	Loss: 482.570923
Train Epoche: 2 [1849/96 (1926%)]	Loss: 482.189209
Train Epoche: 2 [1850/96 (1927%)]	Loss: 0.954049
Train Epoche: 2 [1851/96 (1928%)]	Loss: 484.204376
Train Epoche: 2 [1852/96 (1929%)]	Loss: 225.783661
Train Epoche: 2 [1853/96 (1930%)]	Loss: 171.416351
Train Epoche: 2 [1854/96 (1931%)]	Loss: 144.237671
Train Epoche: 2 [1855/96 (1932%)]	Loss: 291.673035
Train Epoche: 2 [1856/96 (1933%)]	Loss: 257.506714
Train Epoche: 2 [1857/96 (1934%)]	Loss: 35.511337
Train Epoche: 2 [1858/96 (1935%)]	Loss: 99.534500
Train Epoche: 2 [1859/96 (1936%)]	Loss: 24.688202
Train Epoche: 2 [1860/96 (1938%)]	Loss: 15.692994
Train Epoche: 2 [1861/96 (1939%)]	Loss: 48.83

Train Epoche: 2 [2012/96 (2096%)]	Loss: 8.717880
Train Epoche: 2 [2013/96 (2097%)]	Loss: 255.301102
Train Epoche: 2 [2014/96 (2098%)]	Loss: 99.241745
Train Epoche: 2 [2015/96 (2099%)]	Loss: 79.984276
Train Epoche: 2 [2016/96 (2100%)]	Loss: 167.752243
Train Epoche: 2 [2017/96 (2101%)]	Loss: 573.900696
Train Epoche: 2 [2018/96 (2102%)]	Loss: 15.572942
Train Epoche: 2 [2019/96 (2103%)]	Loss: 63.484024
Train Epoche: 2 [2020/96 (2104%)]	Loss: 48.618473
Train Epoche: 2 [2021/96 (2105%)]	Loss: 120.083748
Train Epoche: 2 [2022/96 (2106%)]	Loss: 3.806166
Train Epoche: 2 [2023/96 (2107%)]	Loss: 0.947203
Train Epoche: 2 [2024/96 (2108%)]	Loss: 142.874649
Train Epoche: 2 [2025/96 (2109%)]	Loss: 573.958618
Train Epoche: 2 [2026/96 (2110%)]	Loss: 35.366634
Train Epoche: 2 [2027/96 (2111%)]	Loss: 24.264486
Train Epoche: 2 [2028/96 (2112%)]	Loss: 223.982513
Train Epoche: 2 [2029/96 (2114%)]	Loss: 196.146942
Train Epoche: 2 [2030/96 (2115%)]	Loss: 323.290741
Train Epoche: 2 [2031/96 (2116%)]	Loss: 288.

In [23]:
def test_times_cuda(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model(data).cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict

def test_times(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            out = model(data)#.cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            #target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict

def test_times_cuda_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model_dynamic.eval()
            #files.listdir(path)
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model_dynamic(data).cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict

def test_times_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            out = model_dynamic(data)#.cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            #target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict


if cuda.lower() =='y':
    total_results = test_times_cuda(test_T)
    total_results_dynamic = test_times_cuda_dynamic(test_T)
else:
    total_results = test_times(test_T)
    total_results_dynamic = test_times_dynamic(test_T)


In [24]:
result_dfs = {}
for raceId, dict in total_results.items():    
    #Auswerten der Vorhersagen aus den Outputdaten des Modells
    A = [x[0][0] for x in list((dict.values()))]
    A_dyn = [x[0][0] for x in list((total_results_dynamic[raceId].values()))]
    y = list(dict.keys())
    y_dyn = list(total_results_dynamic[raceId].keys())
    t = pd.DataFrame(columns = ['target', 'prediction'])
    t_dyn = pd.DataFrame(columns = ['target', 'prediction'])
    t['target'] = y
    t['prediction'] = A
    t_dyn['target'] = y_dyn
    t_dyn['prediction'] = A_dyn
    #print("raceId", raceId)
    temp = split_by_race[raceId]
    #name = split_by_race[raceId].where()
    #sortieren des DataFrames nach den vorhergesagten Positionen, aufsteigend
    end = sqldf.sqldf('''select * from t order by prediction ASC''')
    end.reset_index(inplace = True)
    end_dyn = sqldf.sqldf('''select * from t_dyn order by prediction ASC''')
    end_dyn.reset_index(inplace = True)
    #Da DF nun nach prediction aufsteigend sortiert ist, kann veränderter Index als predictete Position gesetzt werden
    end.rename(columns = {'index': 'predicted_position'},inplace = True)
    end_dyn.rename(columns = {'index': 'predicted_position'},inplace = True)
    #zur Übersichtlichkeit wird nun der endgültige DF nach den richtigen Positionen (target) sortiert (aufsteigend)
    end = sqldf.sqldf('''select * from end order by target ASC''')
    end_dyn = sqldf.sqldf('''select * from end_dyn order by target ASC''')
    end['predicted_position'] = end['predicted_position']+1
    end_dyn['predicted_position'] = end_dyn['predicted_position']+1
    #pred_name = sqldf.sqldf("""select 
    #            distinct
    #            d.driver_fullname as name,
    #            t.predicted_position as pred,
    #            d.podium_position as target_position
    #            from 
    #            end t inner join temp d
    #            on t.predicted_position = d.podium_position""")
    end["driver_pred"] = 0
    end["driver_target"] = 0
    end_dyn["driver_pred"] = 0
    end_dyn["driver_target"] = 0
    for pos in temp.podium_position.unique():
        
        name = temp.where(temp.podium_position == pos).dropna(how = "all")
        #print(name)
        name_ = name["driver_fullname"][list(name.index)[0]]
        idx_target = end.where(end.target == pos).dropna(how = "all").index
        idx_pred = end.where(end.predicted_position == pos).dropna(how = "all").index
        end.loc[idx_target,"driver_target"] = name_
        end.loc[idx_pred,"driver_pred"] = name_
        
        idx_target = end_dyn.where(end_dyn.target == pos).dropna(how = "all").index
        idx_pred = end_dyn.where(end_dyn.predicted_position == pos).dropna(how = "all").index
        end_dyn.loc[idx_target,"driver_target"] = name_
        end_dyn.loc[idx_pred,"driver_pred"] = name_
        
    #umstellen der Spaltenreihenfolge
    '''end = end[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]'''
    end = end[['target', 'predicted_position', 'prediction']]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction']]
    result_dfs[raceId] = end
    r = str(raceId)+'_dynamic'
    result_dfs[r] = end_dyn

In [25]:
for key, value in result_dfs.items():
    print('RaceId:',key)
    print(value,'\n')

RaceId: 977
    target  predicted_position  prediction
0      1.0                   1    4.608701
1      2.0                   2    6.688302
2      3.0                   5    7.273826
3      4.0                   3    7.031137
4      5.0                   4    7.176562
5      6.0                   8   10.772938
6      7.0                   7   10.440473
7      8.0                   6   10.280284
8      9.0                   9   14.760722
9     10.0                  10   15.240193
10    11.0                  11   16.974724
11    12.0                  14   18.260279
12    13.0                  12   17.071960
13    14.0                  13   18.122776
14    15.0                  17   20.587730
15    16.0                  15   19.672127
16    20.0                  16   20.417309 

RaceId: 977_dynamic
    target  predicted_position  prediction
0      1.0                  17    0.049432
1      2.0                   5    0.009425
2      3.0                  11    0.032367
3      4.0          

In [26]:
model_dynamic

NetzDynamic(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=150, bias=True)
    (1): Linear(in_features=150, out_features=180, bias=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=180, out_features=190, bias=True)
    (4): Linear(in_features=190, out_features=120, bias=True)
    (5): Linear(in_features=120, out_features=100, bias=True)
    (6): Linear(in_features=100, out_features=70, bias=True)
    (7): Linear(in_features=70, out_features=30, bias=True)
    (8): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [27]:
model

Netz(
  (fc1): Linear(in_features=10, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=180, bias=True)
  (fc3): Linear(in_features=180, out_features=190, bias=True)
  (fc4): Linear(in_features=190, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=70, bias=True)
  (fc7): Linear(in_features=70, out_features=30, bias=True)
  (fc8): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

### Anhang

nogo_columns_hannah_kacke = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']

In [ ]:
#test_netz = {'relu1':['linear',53,150], 'relu2':['linear',150,180], 'relu3':['dropout', 180,100],'relu4':['linear',180,30],'relu5':['linear',30,1]}

model_dynamic = NetzDynamic(opt_combination)

optimizer = optim.Adam(model_dynamic.parameters(), lr = 0.001)     

def train_cuda(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
        
    #random.shuffle(train_data)
def train_(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)#.cuda()
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
        

for epoch in range(1,3):
    train_(epoch)  
        